In [43]:
import numpy as np
from skimage.feature import peak_local_max

def normalize(smap):
    # REF: https://ieeexplore.ieee.org/abstract/document/5651381
    # This operator uses the global
    # maximum of each map to obtain a dynamic range between
    # 0 and 1
   
    if smap.min() <0 :
        sal_map=smap+np.abs(smap.min())
    else:
        sal_map=smap
        
    sal_map=sal_map/sal_map.max()
    
    return sal_map
    
def normalized_and_sum(smap1, smap2):
    # REF: https://ieeexplore.ieee.org/abstract/document/5651381
    
    n_smap1 = normalize(smap1)
    n_smap2 = normalize(smap2)
    
    raw_sum_map = n_smap1 + n_smap2
    
    outmap = normalize(raw_sum_map)
    
    return outmap


def normalized_and_maximum(smap1, smap2):
    # REF: https://ieeexplore.ieee.org/abstract/document/5651381
    
    n_smap1 = normalize(smap1)
    n_smap2 = normalize(smap2)
    
    outmap = np.maximum(n_smap1, n_smap2)

    return outmap
    
def simple_pixel_wise_product(map_a, map_b):
    comb_map=map_a*map_b
    
    #normalize
    comb_map=normalize(comb_map)
    
    return comb_map

def coherent_normalization_with_weighted_sum(smap1, smap2, alpha=0.5, beta=1.0):
    # REF: https://ieeexplore.ieee.org/document/6605606
    
    outmap = (1 - alpha)*smap1 + alpha*smap2 + beta*( smap1*smap2)
    
    return outmap

def average_local_maxima(smap):
    local_max_loc=peak_local_max(smap)
    
    #all local maxima
    all_local_max = smap[local_max_loc[:,0], local_max_loc[:,1]]
    
    # drop the absolute maximum
    other_local_max = np.delete(all_local_max, np.argmax(all_local_max))
    
    return np.mean(other_local_max)

def global_nonlinear_amplification(smap1, smap2):
    # original REF: http://authors.library.caltech.edu/71459/1/161_1.pdf
    # REF: https://ieeexplore.ieee.org/abstract/document/5651381
    # REF for local maxima: https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_peak_local_max.html
    
    n_smap1 = normalize(smap1)
    n_smap2 = normalize(smap2)
    
    # follows notation of Le Meur 
    M_1 = n_smap1.max()
    m_1 = average_local_maxima (smap1)
    
    M_2 = n_smap2.max()
    m_2 = average_local_maxima (smap2)
    
    
    outmap = ( n_smap1 * (M_1-m_1)**2 ) + ( n_smap2 * (M_2-m_2 )**2 )
    
    return outmap


In [44]:
import os
import subprocess 

gbvs_bin = "/home/serov/code/cpp/GBVS_fork/build/gbvs_main"

def get_gbvs_cpp(input_filename):
    output_filename = input_filename.replace('image.png', 'gbvs.exr')
    gbvs_cmd = gbvs_bin + " -i " + input_filename + " -o " + output_filename
    p = subprocess.Popen(gbvs_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if p.wait() == 0:
        return cv2.imread(output_filename, flags=(cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH) )
    else:
        raise Exception("Couldn't create GBVS saliency map.")

def create_gbvs_cpp(input_filename):
    output_filename = input_filename.replace('image.png', 'gbvs.exr')
    gbvs_cmd = gbvs_bin + " -i " + input_filename + " -o " + output_filename
    p = subprocess.Popen(gbvs_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if p.wait() == 0:
        print("Created gbvs map at:", output_filename)
    else:
        raise Exception("Couldn't create GBVS saliency map.")


In [45]:
def overlay_image_and_saliency(im: np.ndarray,
                               saliency: np.ndarray,
                               resize_factor: float) -> np.ndarray:

    im_resize = cv2.resize(im, None, fx=resize_factor, fy=resize_factor, interpolation=cv2.INTER_AREA)
    saliency_resize = cv2.resize(saliency, None, fx=resize_factor, fy=resize_factor, interpolation=cv2.INTER_AREA)
    if im_resize.dtype != np.uint8:
        im_resize = np.asarray(im_resize, dtype=np.uint8)
    if saliency_resize.dtype != np.uint8:
        saliency_resize = np.asarray(saliency_resize * 255, dtype=np.uint8) # 
    if len(im_resize.shape) != len(saliency_resize.shape):
        if len(saliency_resize.shape) == 2:  # grayscale
            saliency_resize = cv2.cvtColor(saliency_resize, cv2.COLOR_GRAY2BGR)  # convert to color
        elif len(saliency_resize).shape == 3:  # color
            saliency_resize = cv2.cvtColor(saliency_resize, cv2.COLOR_BGR2GRAY)  # convert to grayscale
    alpha = 0.5
    beta = 0.5
    gamma = 0
    return cv2.addWeighted(im_resize, alpha, saliency_resize, beta, gamma)


def overlay_image_and_saliency_as_heatmap(im: np.ndarray,
                                          saliency: np.ndarray,
                                          resize_factor: float) -> np.ndarray:

    im_resize = cv2.resize(im, None, fx=resize_factor, fy=resize_factor, interpolation=cv2.INTER_AREA)
    saliency_resize = cv2.resize(saliency, None, fx=resize_factor, fy=resize_factor, interpolation=cv2.INTER_AREA)
    if im_resize.dtype != np.uint8:
        im_resize = np.asarray(im_resize, dtype=np.uint8)
    if saliency_resize.dtype != np.uint8:
        saliency_resize = np.asarray(saliency_resize * 255, dtype=np.uint8)
    if len(im_resize.shape) != len(saliency_resize.shape):
        if len(saliency_resize.shape) == 2:  # grayscale
            saliency_resize = cv2.cvtColor(saliency_resize, cv2.COLOR_GRAY2BGR)  # convert to color
        elif len(saliency_resize).shape == 3:  # color
            saliency_resize = cv2.cvtColor(saliency_resize, cv2.COLOR_BGR2GRAY)  # convert to grayscale

    saliency_resize = cv2.applyColorMap(saliency_resize, cv2.COLORMAP_JET)
    
    alpha = 0.7
    beta = 0.3
    gamma = 0
    return cv2.addWeighted(im_resize, alpha, saliency_resize, beta, gamma)


In [46]:

def mean_absolute_error(saliency: np.ndarray,
                        groundtruth: np.ndarray) -> float:
    assert(saliency.shape == groundtruth.shape)
    height, width = saliency.shape
    # print("max saliency    ", np.max(saliency))
    # print("max groundtruth ", np.max(groundtruth))
    dtype_saliency = saliency.dtype
    return 1 / (height * width) * np.sum(saliency - np.asarray(groundtruth / 255.0, dtype=dtype_saliency))


In [47]:
from typing import Tuple
import cv2

def cv2_put_multi_line_text(im: np.ndarray,
                            text: str,
                            center: Tuple,
                            color: Tuple,
                            font=cv2.FONT_HERSHEY_COMPLEX,
                            font_scale=0.5,
                            thickness=1) -> None:
    text_size, _ = cv2.getTextSize("dummy_text", font, font_scale, thickness)
    line_height = text_size[1] + 5
    y0 = int(center[1])
    for i, text_line in enumerate(text.split('\n')):
        y = y0 + i * line_height
        cv2.putText(im, text_line, (int(center[0]), y),
                    cv2.FONT_HERSHEY_COMPLEX, font_scale, color[::-1], thickness, cv2.LINE_AA)


In [48]:
class RunningStats():
    """ 
    see https://github.com/liyanage/python-modules/blob/master/running_stats.py
    """
    def __init__(self):
        self.n = 0
        self.old_m = 0
        self.new_m = 0
        self.old_s = 0
        self.new_s = 0

    def clear(self):
        self.n = 0

    def add_sample(self, x):
        self.n += 1

        if self.n == 1:
            self.old_m = self.new_m = x
            self.old_s = 0
        else:
            self.new_m = self.old_m + (x - self.old_m) / self.n
            self.new_s = self.old_s + (x - self.old_m) * (x - self.new_m)

            self.old_m = self.new_m
            self.old_s = self.new_s

    def mean(self):
        return self.new_m if self.n else 0.0

    def variance(self):
        return self.new_s / (self.n - 1) if self.n > 1 else 0.0

    def standard_deviation(self):
        return np.sqrt(self.variance())


In [56]:
import glob

# TODO save MAE stats for every sample to find outliers and investigate distribution of data

width = 1600
height = 900
display_image = True
save_images = True
# get all images and birth masks
base_data_path = "/data/datasets/nuscenes_results/birth_frames_and_masks"

image_filenames = sorted(glob.glob(base_data_path + os.sep + "*image*", recursive=False))
mask_filenames = sorted(glob.glob(base_data_path + os.sep + "*mask*", recursive=False))
gbvs_filenames = sorted(glob.glob(base_data_path + os.sep + "*gbvs*", recursive=False))

assert(len(image_filenames) == len(mask_filenames) and len(gbvs_filenames) == len(mask_filenames))

# create spectral residual object
spectral_residual_height = 8
spectral_residual_width = 256
spectral_residual = cv2.saliency.StaticSaliencySpectralResidual_create()
spectral_residual.setImageHeight(spectral_residual_height)
spectral_residual.setImageWidth(spectral_residual_width)

# prepare object birth histogram
baseline_histogram_filename = "/data/datasets/nuscenes_results/object_birth_heatmaps/birth_heatmap_0849.npy"
baseline_birth_histogram = np.load(baseline_histogram_filename)
baseline_birth_histogram = normalize(baseline_birth_histogram)

# saliency maps to be evaluated
saliency_types = ['baseline', 'gbvs', 'spectral_residual_h8w256']
# Combinations?
saliency_results = {}
for saliency_type in saliency_types:
    saliency_results[saliency_type] = RunningStats()

for image_filename, mask_filename, gbvs_filename in zip(image_filenames, mask_filenames, gbvs_filenames):
    img = cv2.imread(image_filename)
    mask = cv2.imread(mask_filename, flags=cv2.IMREAD_GRAYSCALE)
    gbvs = cv2.imread(gbvs_filename, flags=(cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH))
    (success, saliency_spectral_residual) = spectral_residual.computeSaliency(img)
    if not success:
        print("Couldn't create saliency for ", image_filename)
        continue
    
    img_and_gbvs = overlay_image_and_saliency_as_heatmap(img, gbvs, 1.0)
    img_and_baseline = overlay_image_and_saliency_as_heatmap(img, baseline_birth_histogram, 1.0)
    # saliency_spectral_residual = cv2.cvtColor((saliency_spectral_residual*255).astype(np.uint8), cv2.COLOR_BGR2GRAY)
    img_and_spectral_residual = overlay_image_and_saliency_as_heatmap(img, normalize(saliency_spectral_residual), 1.0)

    mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    img_and_gbvs_and_mask = cv2.addWeighted(img_and_gbvs, 0.7, mask_bgr, 0.3, 0.0)
    img_and_baseline_and_mask = cv2.addWeighted(img_and_baseline, 0.7, mask_bgr, 0.3, 0.0)
    img_and_spectral_residual_and_mask = cv2.addWeighted(img_and_spectral_residual, 0.7, mask_bgr, 0.3, 0.0)

    mae_baseline = mean_absolute_error(baseline_birth_histogram, mask)
    mae_gbvs = mean_absolute_error(gbvs, mask)
    mae_spectral_residual = mean_absolute_error(saliency_spectral_residual, mask)#

    saliency_results['baseline'].add_sample(mae_baseline)
    saliency_results['gbvs'].add_sample(mae_gbvs)
    saliency_results['spectral_residual_h8w256'].add_sample(mae_spectral_residual)

    mae_baseline = mean_absolute_error(baseline_birth_histogram, mask)
    cv2_put_multi_line_text(img_and_gbvs_and_mask, str(mae_gbvs), (width/2, height/2), (0, 0, 0))
    cv2_put_multi_line_text(img_and_baseline_and_mask, str(mae_baseline), (width/2, height/2), (0, 0, 0))
    cv2.imshow("input img and gbvs overlayed", img_and_spectral_residual_and_mask)
    key = cv2.waitKey(0)
    print("MEAN base {} gbvs {} spectral {}".format(
        saliency_results['baseline'].mean(),
        saliency_results['gbvs'].mean(),
        saliency_results['spectral_residual_h8w256'].mean()))
    print("STD  base {} gbvs {} spectral {}\n".format(
        saliency_results['baseline'].standard_deviation(),
        saliency_results['gbvs'].standard_deviation(),
        saliency_results['spectral_residual_h8w256'].standard_deviation()))
    if key == 27:  # if ESC is pressed, exit.
        cv2.destroyAllWindows()
        break

cv2.destroyAllWindows()
    


MEAN base 0.06495029233779233 gbvs 0.13824206814236112 spectral 0.04959339735243056
STD  base 0.0 gbvs 0.0 spectral 0.0

MEAN base 0.0705780701155701 gbvs 0.148108935546875 spectral 0.08864437662760416
STD  base 0.00795887965935526 gbvs 0.01395385770160056 spectral 0.05522642451490118

MEAN base 0.0744056164118664 gbvs 0.1825321361400463 spectral 0.0873639087818287
STD  base 0.0086961034185964 gbvs 0.06043364368968153 spectral 0.03911390770636748

MEAN base 0.07475723678223677 gbvs 0.21065308430989582 spectral 0.09096859130859375
STD  base 0.007135079349060313 gbvs 0.0748195680529257 spectral 0.03273998770896557

MEAN base 0.07350015344890344 gbvs 0.2173189214409722 spectral 0.09090468424479167
STD  base 0.006788468938042098 gbvs 0.06648791428530428 spectral 0.028354021177805302

MEAN base 0.07328107937482937 gbvs 0.2176241482204861 spectral 0.08582279459635417
STD  base 0.0060954580728439425 gbvs 0.05947329801793045 spectral 0.028250911984474457

MEAN base 0.07383352646477646 gbvs 0.2